# S3W9D5: 检索评估与优化 (Retrieval Evaluation)

**今日目标**：
建立一套自动化的 RAG 评测流程，计算 **Hit Rate (命中率)**，并学会如何分析检索失败的案例（Bad Cases）。

## 1. 理论知识讲解 (CS Principle)

在传统机器学习中，我们看重 Accuracy / F1-Score。但在 RAG 检索阶段，我们主要关注 **Hit Rate@K**。

### 核心指标：Hit Rate@K

* **定义**：对于给定的问题，正确的文档是否出现在了前 K 个检索结果中？
* **公式**：

$$\text{Hit Rate@K} = \frac{\text{Number of queries with correct doc in Top-K}}{\text{Total number of queries}}$$

* **场景**：
* 如果 `K=3`，只要正确答案在第 1、2 或 3 位，都算成功（Hit）。
* 如果正确答案在第 4 位，或者根本没搜到，算失败（Miss）。

## 2. 代码实现 (The Evaluator)

我们要编写一个脚本，让它自动问 RAG 系统 10 个问题，并检查它回回来的文档对不对。

### Step 1: 准备“真值数据集” (Ground Truth)

基于昨天写入的 `penal_code.txt`（刑法片段），我们构造一组测试题。

In [1]:
# 测试集结构：问题 + 预期的关键词/法条
# 只要检索回来的文本里包含 "expected_content"，我们就认为检索成功
test_dataset = [
    {
        "query": "偷东西金额巨大怎么判？",
        "expected_content": "盗窃公私财物", 
        "expected_article": "264" # 辅助验证
    },
    {
        "query": "入户盗窃怎么定罪？",
        "expected_content": "入户盗窃",
        "expected_article": "264"
    },
    {
        "query": "诈骗罪最高能判死刑吗？",
        "expected_content": "诈骗公私财物", # 检索结果应该包含诈骗罪的法条
        "expected_article": "266"
    },
    {
        "query": "把人打成重伤怎么判？",
        "expected_content": "故意伤害他人身体",
        "expected_article": "234"
    },
    {
        "query": "什么情况下会判死刑？",
        "expected_content": "致人死亡或者以特别残忍手段致人重伤造成严重残疾",
        "expected_article": "234" 
    }
]

### Step 2: 编写评估函数

我们将复用 `src/rag/vector_db.py` 中的 `VectorDB` 类。

In [ ]:
import sys
import os
# 确保能导入 src
sys.path.append(os.path.abspath("../../"))

from src.rag.vector_db import VectorDB

def evaluate_retrieval(test_set, top_k=3):
    # 1. 加载数据库 (注意路径要对)
    # 假设我们在项目根目录下运行，或者是用绝对路径
    # 这里需要指向昨天 ingest.py 生成的那个文件夹
    db_path = "../../data/chroma_db_data" 
    vector_db = VectorDB(persist_path=db_path)
    
    hits = 0
    total = len(test_set)
    bad_cases = []

    print(f"🚀 Starting Evaluation (Top-K={top_k})...")
    print("-" * 50)

    for case in test_set:
        query = case["query"]
        expected = case["expected_content"]
        
        # 2. 执行检索
        results = vector_db.search(query, top_k=top_k)
        
        # 3. 判定是否命中
        is_hit = False
        retrieved_texts = []
        
        for res in results:
            text = res["text"]
            retrieved_texts.append(text[:20] + "...") # 记录一下以便调试
            # 简单的字符串包含匹配 (Fuzzy Matching)
            if expected in text:
                is_hit = True
                break
        
        # 4. 统计
        if is_hit:
            hits += 1
            print(f"✅ Hit: {query}")
        else:
            print(f"❌ Miss: {query}")
            bad_cases.append({
                "query": query,
                "expected": expected,
                "retrieved": retrieved_texts
            })

    # 5. 计算分数
    score = hits / total
    print("-" * 50)
    print(f"📊 Final Score: {score:.2%} ({hits}/{total})")
    
    return bad_cases

# 运行评估
bad_cases = evaluate_retrieval(test_dataset, top_k=3)

Initializing Embedding Model: BAAI/bge-small-zh-v1.5...
Attempting to load from local cache (Offline Mode)...
✅ Successfully loaded from local cache.
🚀 Starting Evaluation (Top-K=3)...
--------------------------------------------------
✅ Hit: 偷东西金额巨大怎么判？
✅ Hit: 入户盗窃怎么定罪？
✅ Hit: 诈骗罪最高能判死刑吗？
✅ Hit: 把人打成重伤怎么判？
✅ Hit: 什么情况下会判死刑？
--------------------------------------------------
📊 Final Score: 100.00% (5/5)


## 3. 深度分析：Bad Case Analysis

如果你的分数不是 100%，不要慌，**Bad Case 才是最有价值的**。请检查 `bad_cases` 列表。

**常见失败原因及 CS 对策**：

1. **关键词不匹配 (Lexical Gap)**：
    * *Case*: 用户问“**骗子**怎么判”，法条里只有“**诈骗**”。
    * *Reason*: 虽然 Embedding 应该能处理近义词，但如果模型太小（如 `bge-small`），有时候对口语词的理解不够。
    * *Fix*: **Query Rewrite (查询重写)**。在检索前，先用 LLM 把用户的口语改写成法言法语（Week 10 会讲）。


2. **切分断裂 (Chunking Fragmentation)**：
    * *Case*: 用户问“情节严重判几年？”，答案在法条的后半段，但前半段（定义罪名）被切到上一个 Chunk 去了，导致后半段的 Chunk 只有刑期，没有罪名，语义不完整。
    * *Fix*: 增加 **Chunk Overlap (重叠率)**，或者使用 **Parent Document Retriever**（检索到小块，返回大块）。


3. **距离失效 (Distance Failure)**：
    * *Case*: Top-1 的分数是 0.6，Top-2 的正确答案分数是 0.59。
    * *Reason*: 向量空间的挤压。
    * *Fix*: 增大 `Top-K`（比如检索 10 个），然后引入 **Rerank (重排序模型)** 进行二次精排。

## 4. 今日 LeetCode 练习

在做评估时，我们经常要用到 **Hash Map (哈希表)** 来快速查找预期答案是否存在，或者统计出现频率。

我在你的未做列表中为你精选了这道题：

### 🎯 [169. 多数元素 (Majority Element)](https://leetcode.cn/problems/majority-element/)

* **难度**：Easy
* **状态**：在你的列表中 (题号 169)。
* **CS 核心**：**Boyer-Moore 投票算法**。
* **题目描述**：给定一个大小为 n 的数组，找到其中的多数元素。多数元素是指在数组中出现次数 **大于** `⌊ n/2 ⌋` 的元素。

**为什么选这道题？**
这道题有一个普通的解法（哈希表， 空间）和一个**天才般的解法**（投票法， 空间）。作为算法工程师，掌握投票法（Voting Algorithm）对于理解集成学习（Ensemble Learning）和 RAG 中的多路召回投票机制非常有帮助。

**思路提示 (投票法)**：

* 假设每个数字代表一个候选人。
* 既然多数元素超过半数，如果你让大家“一换一”（一个众数和一个非众数抵消），**最后剩下的一定是众数**。

请尝试实现并提交。

## 5. 今日任务总结

* [ ] 编写了 `S3W9D5_Eval.ipynb`，建立了自动化评测流程。
* [ ] 运行了评估，计算出了你当前 RAG 系统的 Hit Rate。
* [ ] 分析了 Bad Cases，理解了 Embedding 和 Chunking 的局限性。
* [ ] 完成了 LeetCode 169。

**预告**：
明天（**Day 6**），我们将进行 **Web 界面集成**。
我们将使用 **Streamlit**，把这几天做的所有模块（Embedding, VectorDB, RAG Logic）封装成一个网页应用。到时候，你就可以把这个链接发给朋友演示你的作品了！